In [1]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    !git clone https://github.com/deepmind/pycolab.git
    !git clone https://github.com/nicoladainese96/RelationalModule.git
    !pip install pycolab

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from RelationalModule import ActorCritic, ControlActorCritic
from RelationalModule import train_agent as train
from RelationalModule import utils
from importlib import reload

In [3]:
reload(train)

<module 'RelationalModule.train_agent' from '/home/nicola/Nicola_unipd/MasterThesis/RelationalDeepRL/RelationalModule/train_agent.py'>

In [4]:
GRID_SIZE = 7
game_params = dict(grid_size=GRID_SIZE,
                solution_length=[0],
                num_forward = [0], # number of distractors
                num_backward=[0], # just set to 0 for now
                branch_length=1, # length of forward distractors
                max_num_steps = 120
               )

In [5]:
HPs = dict(action_space=4, lr=0.003, gamma=0.99, TD=True, twin=True, tau=0.2, n_steps=40,
           n_kernels=96, vocab_size = 117, n_dim=12, n_features=64, n_heads=4, n_attn_modules=2, 
           n_linears=4, max_pool=False, linear_size=GRID_SIZE+2)
if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'

print('device: ', HPs['device'])  

# Relational Agent
agent = ActorCritic.BoxWorldA2C(**HPs)

control_HPs = dict(action_space=4, lr=0.003, gamma=0.99, TD=True, twin=True, tau=0.2, n_steps=40, linear_size=GRID_SIZE+2)
           
# Control Agent
control_agent = ControlActorCritic.ControlA2C(**control_HPs)

device:  cpu
========== A2C HyperParameters ==========
Discount factor:  0.99
Learning rate:  0.003
Action space:  4
Temporal Difference learning:  True
Twin networks:  True
Update critic target factor:  0.2
n_steps for TD:  40
Device used:  cpu


========== A2C Architecture ==========
Actor architecture: 
 BoxWorldActor(
  (boxnet): BoxWorldNet(
    (net): Sequential(
      (0): ExtractEntities(
        (embed): Embedding(117, 12)
        (net): Sequential(
          (0): Conv2d(12, 48, kernel_size=(2, 2), stride=(1, 1))
          (1): ReLU()
          (2): Conv2d(48, 96, kernel_size=(2, 2), stride=(1, 1))
          (3): ReLU()
        )
      )
      (1): RelationalModule(
        (net): Sequential(
          (0): PositionalEncoding(
            (projection): Linear(in_features=98, out_features=64, bias=True)
          )
          (1): AttentionBlock(
            (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
        

In [6]:
# Random Agent

class RandomAgent():
    def __init__(self, n_actions):
        self.n_actions = n_actions
    
    def get_action(self,state, *args, **kwargs):
        a = np.random.choice(self.n_actions)
        log_prob = np.log(1./self.n_actions) # just because it's the standard output of the other agent
        return a, log_prob
    
    def update(self, *args):
        return

rnd_agent = RandomAgent(4)

In [7]:
%%time
results = train.train_boxworld(agent, game_params, n_episodes=5000, 
                               max_steps=game_params['max_num_steps'], return_agent=True, mask=True)
score, asymptotic_score, asymptotic_std, trained_agent, time_profile = results

/home/nicola/anaconda3/envs/torch/lib/python3.7/site-packages/pycolab/ascii_art.py:318: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)


distribution:  tensor([[0.0469, 0.3524, 0.4029, 0.1978]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.1030, 0.2861, 0.2850, 0.3260]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0565, 0.2489, 0.5387, 0.1558]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0718, 0.3318, 0.2542, 0.3421]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0868, 0.1520, 0.5506, 0.2107]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0906, 0.4149, 0.3017, 0.1927]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0637, 0.2144, 0.3684, 0.3534]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.1399, 0.3144, 0.2605, 0.2852]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0675, 0.3322, 0.4236, 0.1767]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0936, 0.3682, 0.4459, 0.0923]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0500, 0.3600, 0.2992, 0.2909]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0692, 0.4174, 0.3606, 0.1528]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.04

V1.shape:  torch.Size([47])
V1:  tensor([[ 0.5859],
        [ 0.2206],
        [ 0.3176],
        [ 0.3007],
        [ 0.2747],
        [ 0.1583],
        [ 0.6236],
        [ 0.4317],
        [-0.7255],
        [-0.3691],
        [ 0.4405],
        [ 0.1400],
        [-0.3781],
        [ 0.4212],
        [ 0.0704],
        [ 0.0202],
        [ 0.1691],
        [ 0.9707],
        [-0.2781],
        [ 0.1818],
        [ 0.3323],
        [-0.4326],
        [-1.0807],
        [-0.0749],
        [-0.7235],
        [-0.0991],
        [ 0.5995],
        [-0.1823],
        [-0.2194],
        [ 0.3198],
        [-0.1203],
        [-0.6519],
        [ 0.2617],
        [ 0.3949],
        [-0.0048],
        [-0.3949],
        [ 0.1856],
        [-0.7357],
        [-0.0497],
        [ 0.1285],
        [-0.7248],
        [ 0.3275],
        [-0.4716],
        [ 0.9068],
        [-0.1003],
        [-0.2856],
        [-0.0735]], grad_fn=<AddmmBackward>)
Updating actor...
V_trg.shape:  torch.Size([47])

distribution:  tensor([[0.0577, 0.5383, 0.2654, 0.1387]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0153, 0.7150, 0.0681, 0.2017]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0191, 0.7043, 0.1336, 0.1430]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0360, 0.6718, 0.1857, 0.1065]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0168, 0.6997, 0.1283, 0.1552]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0595, 0.4326, 0.3744, 0.1334]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0260, 0.5345, 0.1099, 0.3295]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0416, 0.4727, 0.1134, 0.3723]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0399, 0.7494, 0.1220, 0.0886]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0950, 0.3664, 0.1992, 0.3394]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0641, 0.4299, 0.2522, 0.2538]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0490, 0.5777, 0.2180, 0.1553]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.02

V_trg.shape (after critic):  torch.Size([121])
V_trg.shape (after sum):  torch.Size([121])
V_trg.shape (after squeeze):  torch.Size([121])
V_trg.shape (after squeeze):  tensor([ 0.2570,  0.1100, -0.0914,  0.2078, -0.2322,  0.0607,  0.0584, -0.1715,
         0.4650,  0.1238, -0.2338, -0.1211,  0.7462,  0.1060,  0.1926,  0.0347,
         0.2076,  0.3013, -0.0686,  0.3416, -0.2913, -0.1524,  0.2432,  0.4209,
         0.2129, -0.1509,  0.3038, -0.0232,  0.0351, -0.3811,  0.4732,  0.0496,
         0.2777, -0.0486,  0.3886,  0.2561,  0.5540,  0.0395,  0.0224,  0.3976,
         0.1485,  0.1888, -0.4534,  0.4603,  0.5567,  0.0235, -0.0297,  0.5398,
         0.3063,  0.1360, -0.1075,  0.2464, -0.2688,  0.4652,  0.3323,  0.6300,
         0.2728,  0.1424, -0.2156,  0.2677,  0.1033,  0.4736, -0.1904, -0.0161,
         0.0779, -0.0126,  0.2039, -0.0159,  0.1423,  0.3091,  0.2807, -0.0091,
        -0.1826,  0.1870,  0.4205, -0.0306,  0.6309,  0.3488,  0.1520, -0.0734,
        -0.1420,  0.0310,  0.50

...

V_trg.shape (after critic):  torch.Size([121])
V_trg.shape (after sum):  torch.Size([121])
V_trg.shape (after squeeze):  torch.Size([121])
V_trg.shape (after squeeze):  tensor([ 0.5738,  0.6104,  0.5511,  0.6906,  0.8225,  0.5154,  0.8526,  0.7682,
         0.5376,  0.6742,  0.9454,  0.6130,  0.6713,  1.0631,  0.7417,  0.8446,
         0.8679,  0.6169,  1.0472,  0.2958, -0.0696,  0.7297,  0.7050,  0.6661,
         0.9077,  1.2574,  0.5316,  0.4586,  0.7473,  0.2964,  0.4462,  0.7954,
         0.4869,  0.6856,  0.3689,  0.4118,  0.4903,  0.8901,  0.4911,  0.5354,
         1.1028,  0.6805,  0.8600,  0.6682,  0.6146,  0.7621,  0.6380,  0.8922,
         0.8848,  0.8053,  0.8377,  0.5395,  0.5027,  0.5657,  0.9564,  0.8364,
         0.1721,  0.2799,  0.9869,  0.2644,  0.2923,  0.7986,  0.4818,  0.4773,
         0.6395,  0.9427,  0.4739,  0.9172,  0.8055,  0.4096,  0.5598,  0.7220,
         0.5975,  0.4529,  1.1029,  0.8274,  1.0230,  1.0984,  0.7205,  0.4684,
         0.2042,  0.7883,  0.96

distribution:  tensor([[0.0037, 0.3403, 0.0047, 0.6513]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0026, 0.0464, 0.0038, 0.9472]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0014, 0.1026, 0.0095, 0.8865]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0098, 0.3318, 0.0335, 0.6249]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0015, 0.0534, 0.0042, 0.9409]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0192, 0.2662, 0.0579, 0.6567]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0035, 0.2827, 0.0633, 0.6506]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0078, 0.3565, 0.0318, 0.6039]], grad_fn=<ExpBackward>)
distribution:  tensor([[2.6220e-04, 1.5005e-02, 4.9477e-04, 9.8424e-01]],
       grad_fn=<ExpBackward>)
distribution:  tensor([[0.0082, 0.5478, 0.0139, 0.4300]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0100, 0.5620, 0.0433, 0.3847]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0048, 0.1266, 0.0016, 0.8670]], grad_fn=<ExpBackward>)
distr

distribution:  tensor([[0.0092, 0.1052, 0.0039, 0.8817]], grad_fn=<ExpBackward>)
distribution:  tensor([[0.0080, 0.0824, 0.0578, 0.8518]], grad_fn=<ExpBackward>)
n_step_rewards.shape:  (121,)
rewards.shape:  (121,)
n_step_rewards:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
rewards:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
bootstrap:  [False

V_trg.shape (after critic):  torch.Size([121])
V_trg.shape (after sum):  torch.Size([121])
V_trg.shape (after squeeze):  torch.Size([121])
V_trg.shape (after squeeze):  tensor([0.7212, 0.7206, 1.1046, 0.8684, 0.9098, 0.6617, 0.4136, 0.1305, 0.4089,
        0.7743, 0.7112, 0.7051, 0.3978, 0.6012, 0.8845, 0.8841, 0.5931, 0.8899,
        0.6572, 0.5082, 0.6362, 1.2112, 0.6973, 0.7983, 0.5948, 0.8058, 1.0111,
        0.5564, 0.5529, 0.5691, 0.6187, 0.3627, 0.8247, 0.7986, 0.7298, 0.6381,
        0.8043, 0.4672, 0.9362, 1.0432, 0.8461, 0.8355, 0.7464, 0.7489, 0.9878,
        0.5901, 0.5978, 0.7857, 1.1877, 0.8350, 0.6563, 0.8072, 0.8075, 0.5594,
        0.7860, 1.1396, 0.3381, 1.1596, 0.8587, 0.8311, 0.7036, 0.6395, 0.9760,
        1.0463, 0.7925, 0.5898, 0.8675, 0.7353, 0.3403, 0.8559, 0.4525, 0.9463,
        0.6892, 0.8039, 0.4847, 0.6992, 0.4918, 1.1734, 0.9529, 1.2067, 1.0017,
        1.1834, 1.1458, 0.4891, 0.9409, 1.0515, 0.9805, 0.6454, 0.9403, 1.2870,
        0.8040, 0.9034, 0.9217,

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(100, len(score))
average_score = np.array([np.mean(score[i:i+100]) for i in range(len(score)-100)])
plt.plot(n_epochs, average_score, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Total reward", fontsize=16)
plt.show()

In [ ]:
save = False
keywords = ['relational', 'residual','unboxed_gem',str(len(control_score))+"-episodes","50-steps"] # example

if colab and save:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    %cd "{parent_dir}"
    !mkdir "{save_dir}"
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score)
    torch.save(trained_agent, save_dir+"agent_"+ID)